# Geometry description with the level set function

Most features of the `ngsxfem` add-on to NGSolve are based on a level set description.

Let $\phi$ be a continuous level set function. 

We define the domains and the interface

$$
\begin{array}{rccc}
  \text{domain:} & \Omega_{-} := \{ \phi < 0 \}, & \quad
  \Omega_{+} := \{ \phi > 0 \}, & \quad
  \Gamma := \{ \phi = 0 \}. \\
  \texttt{ngsxfem} \text{ keyword:} & \texttt{NEG} & 
  \texttt{POS} & 
  \texttt{IF}  
\end{array}
$$

Let us import `Netgen/NGSolve` stuff first:

In [ ]:
import netgen.gui
%gui tk

# ngsolve stuff
from ngsolve import *
# basic geometry features (for the background mesh)
from netgen.geom2d import SplineGeometry
# visualization stuff
from ngsolve.internal import *

We add the basic functionality of the `ngsxfem` package:

In [ ]:
# basic xfem functionality
from xfem import *

## The background mesh
We generate the background mesh of the domain

In [ ]:
square = SplineGeometry()
square.AddRectangle([-1.5,-1.5],[1.5,1.5],bc=1)
mesh = Mesh (square.GenerateMesh(maxh=0.5, quad_dominated=False))

On the background mesh we define the level set function. 

In the visualization we distinguish the negative and the positive part of the level set values corresponding to the subdomains that are described.

In [ ]:
levelset = (sqrt(sqrt(x**4+y**4)) - 1.0)
Draw(levelset,mesh,"levelset")
visoptions.mminval = 0.0
visoptions.mmaxval = 0.0
visoptions.deformation = 0
visoptions.autoscale = 0
Redraw()

## Geometry approximation through level set approximation
* Arbitrary level set functions can be difficult to handle (cut topology)
* To obtain something feasible, we interpolate into the space of piecewise linear functions
* The thusly obtained approximation is straight within each (simplex) element
* Simplifies numerical integration
* We will discuss enhancements later ...

In [ ]:
lsetp1 = GridFunction(H1(mesh,order=1))
InterpolateToP1(levelset,lsetp1)
Draw(lsetp1,mesh,"lsetp1")
visoptions.mminval = 0.0
visoptions.mmaxval = 0.0
visoptions.deformation = 0
visoptions.autoscale = 0
Redraw()

* `InterpolateToP1` takes vertex values (in contrast to `Set`)

## Cut Information

In unfitted FEM it is important to know which elements are cut by the interface $ \{ \phi = 0 \} $. These informations are gathered in the cut information class:

In [ ]:
ci = CutInfo(mesh, lsetp1)
# help(CutInfo)

### Marking elements

### `DOMAIN_TYPE`
We can ask for BitArrays corresponding to the differently marked elements 
  * `NEG` : True if $\phi < 0$ everywhere on this element, else False
  * `POS` : True if $\phi > 0$ everywhere on this element, else False
  * `IF` : True if $\phi = 0$ somewhere on this element, else False  

In [ ]:
Draw(BitArrayCF(ci.GetElementsOfType(IF)),mesh,"cut_elements")

In [ ]:
Draw(BitArrayCF(ci.GetElementsOfType(NEG)),mesh,"neg_elements")

In [ ]:
Draw(BitArrayCF(ci.GetElementsOfType(POS)),mesh,"pos_elements")

### `COMBINED_DOMAIN_TYPE`
There are also some predefined combinations of the previous markers:
  
| IF|POS|NEG| combined flag | description    |
|---|---|---|---------------|----------------|
|  0|  0|  0|            NO | False on all elements
|  0|  0|  1|           NEG | True if $\phi < 0$ everywhere on this element, else False   
|  0|  1|  0|           POS | True if $\phi > 0$ everywhere on this element, else False
|  0|  1|  1|         UNCUT | True if $\phi \neq 0$ everywhere on this element, else False   
|  1|  0|  0|            IF | True if $\phi = 0$ somewhere on this element, else False  
|  1|  0|  1|        HASNEG | True if $\phi \leq 0$ somewhere on this element, else False
|  1|  1|  0|        HASPOS | True if $\phi \geq 0$ somewhere on this element, else False
|  1|  1|  1|           ANY | True on all elements
  

In [ ]:
Draw(BitArrayCF(ci.GetElementsOfType(NO)),mesh,"no_elements")
Draw(BitArrayCF(ci.GetElementsOfType(ANY)),mesh,"any_elements")
Draw(BitArrayCF(ci.GetElementsOfType(HASNEG)),mesh,"hasneg_elements")
Draw(BitArrayCF(ci.GetElementsOfType(HASPOS)),mesh,"haspos_elements")
Draw(BitArrayCF(ci.GetElementsOfType(UNCUT)),mesh,"uncut_elements")

## Updating the CutInformation
* level set can be changed with `Update`:

In [ ]:
levelset = sqrt((x-0.5)**2+y**2)-1
Draw(levelset,mesh,"levelset") # overwrite visualization
InterpolateToP1(levelset,lsetp1)
ci.Update(lsetp1)
Redraw()

## Marking with facets 
 * `GetFacetsWithNeighborTypes`
 * `GetElementsWithNeighborFacets`

* Sometimes want to mark some facets depending on their neighbor states (NEG/POS/IF)
* Needed for some stabilizations in unfitted FEM (CutFEM/XFEM/..)
* Facet-marking based on two element-based BitArrays. 
* There are two options to connect two BitArrays A and B:
 * use_and=True (default): the result is True for a facet if (A(left) and B(right)) or (B(left) and A(right))
 * use_and=False     (or): the result is True for a facet if (A(left) or B(right)) or (B(left) or A(right))
 
| A(left) | B(right) | partial result (`use_and=True`) | partial result (`use_and=False`) |
|----------|-----------|-----------------------|------------------------|
| False    | False     | False                 | False                  |
| False    | True      | False                 | True                   |
| True     | False     | False                 | True                   |
| True     | True      | True                  | True                   |
 
* with bnd_val_a and bnd_val_b (default: True) we can prescribe how the BitArray should be interpreted if there is only one neighbor (boundary).

### Example 1:

* All facets that are on the outer boundary of the NEG-domain:

In [ ]:
#help(GetFacetsWithNeighborTypes)
ba_facets = GetFacetsWithNeighborTypes(mesh,
                                       a=ci.GetElementsOfType(NEG),
                                       b=ci.GetElementsOfType(IF),
                                       bnd_val_a=False,
                                       bnd_val_b=False)

* We can also go from facets to elements

In [ ]:
#help(GetElementsWithNeighborFacets)
ba_surround_facets = GetElementsWithNeighborFacets(mesh,ba_facets)
Draw(BitArrayCF(ba_surround_facets), mesh, "surrounding_facets")

### Example 2:

We mark all facets that have at least one neighbor in the NEG-domain.

In [ ]:
ba_facets_NEG = GetFacetsWithNeighborTypes(mesh,
                                           a=ci.GetElementsOfType(NEG),
                                           b=ci.GetElementsOfType(NEG),
                                           bnd_val_a=False,
                                           bnd_val_b=False,
                                           use_and=False)
ba_extended_NEG = GetElementsWithNeighborFacets(mesh,ba_facets_NEG)
Draw(BitArrayCF(ba_extended_NEG), mesh, "extended_NEG")

### Example 3:
We can successively apply this to extend from some part of the domain by direct neighbors, etc ....

In [ ]:
ba_facets_NEG = GetFacetsWithNeighborTypes(mesh,a=ba_extended_NEG,b=ba_extended_NEG,
                                           bnd_val_a=False,bnd_val_b=False,use_and=False)
ba_extended_NEG = GetElementsWithNeighborFacets(mesh,ba_facets_NEG)
Draw(BitArrayCF(ba_extended_NEG), mesh, "extended_NEG")

## Restriction to *active* mesh/`dof`s

### Marking dofs (`GetDofsOfElements`)

* Unfitted FEM: Solve a PDE problem *restricted* to a subdomain, e.g.
 * a fictitious domain method on $\Omega_{-} = \{ \phi < 0 \}$
 * a trace finite element on $\Gamma = \{ \phi = 0\}$
 
* Standard `FESpace` but only a selection of `dof`s (`freedofs`)
* We have helper functions for that as well

In [ ]:
VhG = H1(mesh, order=1, dirichlet=[])
freedofs = VhG.FreeDofs()
freedofs &= GetDofsOfElements(VhG,ci.GetElementsOfType(HASNEG))

* BitArray (`freedofs`) restriction allows to *solve* only on a subset of FEM unknowns
* What to do if you want to set up the matrices only on a selection of elements / unknowns?

### Marking integrators (`definedonelements`)
We can define integrators on a set of elements only:

In [ ]:
integrator = SymbolicBFI(VhG.TrialFunction()*VhG.TestFunction(), 
                         definedonelements=ci.GetElementsOfType(HASNEG))

Or change the *active elements* after definition (for moving domains):

In [ ]:
integrator = SymbolicBFI(VhG.TrialFunction()*VhG.TestFunction())
integrator.SetDefinedOnElements(ci.GetElementsOfType(HASNEG))

### Restricted Bilinearforms
Furthermore, to avoid that matrices are set up for the full standard FESpace we can restrict matrix couplings to elements (and facets for dg terms):

In [ ]:
#help(RestrictedBilinearForm)
a = RestrictedBilinearForm(VhG)
ar = RestrictedBilinearForm(VhG,element_restriction=ci.GetElementsOfType(IF),facet_restriction=None)
a.Assemble()
ar.Assemble()

* This does not change the dimension:

In [ ]:
print(a.mat.height,a.mat.width)
print(ar.mat.height,ar.mat.width)

* But the number on non-zero entries:

In [ ]:
print(len(a.mat.AsVector()))
print(len(ar.mat.AsVector()))